#### Here I am going to try to implement the core of the parsing mechanism using arbitrary data input
#### This code does not implement selection and filtering of data from the unsdg database

In [1]:
# List of variables

# In selected_indicators

# topic
# priority_rank
# priority_order
# goal
# goal.1
# indicator_description
# indicator_id
# series_code
# age_group
# location
# sex
# value_type
# unit
# parse_string_for_indicator
# percent_divisor
# value_judgement

# In unsdg_data_exna

# goal
# target
# indicator.ref
# indicatorid
# indicator.description
# series.code
# series.type
# series.description
# parent.country.or.area.code
# country.or.area.code
# country.or.area.name
# ldc
# lldc
# sids
# frequency
# source.type
# age.group
# location
# sex
# value.type
# unit
# unit.multiplier
# year
# value
# footnote

# Set up globals and packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

current_working_directory = os.getcwd()

#print (current_working_directory)
#print ()
#print ("Done")

c:\Users\Aramsden\Python

Done


In [2]:
# Set up descriptor tables

percent_desc = pd.DataFrame({'percent_lower_bound': [0.00, 0.05, 0.34, 0.68, 0.95],
                             'percent_upper_bound': [0.05, 0.34, 0.68, 0.95, 1.00],
                             'percent_text_desc'  : ["almost none","some","about half","many","almost all"]},
                             index=[0, 1, 2, 3, 4])

comparison_desc = pd.DataFrame({'comparison_lower_bound': [-999999999, -1.0, -0.2, -0.1, 0.1, 0.2, 1.0],
                                'comparison_upper_bound': [        -1, -0.2, -0.1,  0.1, 0.2, 1.0, 999999999],
                                'comparison_text_desc'  : ["very much lesser","much lesser","lesser","about the same","greater","much greater","very much greater"]},
                               index=[0, 1, 2, 3, 4, 5, 6])

#print (percent_desc)
#print ()
#print (comparison_desc)
#print ()
#print ("Done")

   percent_lower_bound percent_text_desc  percent_upper_bound
0                 0.00       almost none                 0.05
1                 0.05              some                 0.34
2                 0.34        about half                 0.68
3                 0.68              many                 0.95
4                 0.95        almost all                 1.00

   comparison_lower_bound comparison_text_desc  comparison_upper_bound
0            -999999999.0     very much lesser                    -1.0
1                    -1.0          much lesser                    -0.2
2                    -0.2               lesser                    -0.1
3                    -0.1       about the same                     0.1
4                     0.1              greater                     0.2
5                     0.2         much greater                     1.0
6                     1.0    very much greater             999999999.0

Done


In [1]:
# Import selected_indicators table
# Assumption: final indicator series for use by topic.csv is in current_working_directory

selected_indicators = pd.read_csv("final_indicator_series_for_use_by_topic_v3.csv", encoding = "ansi")

#selected_indicators.head(3)

NameError: name 'pd' is not defined

In [4]:
# Import sdg_data table (version excluding value NAs)
# Assumption: sdg_data_exna.csv is in current_working_directory

unsdg_data_exna = pd.read_csv("sdg_data_exna.csv", encoding = "ansi")

#unsdg_data_exna.head(3)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (12,13,14,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
# Set up default process input variables

user_name = "Alex"
user_sex = "male"
user_country = "New Zealand"
target_country = "Brazil"
target_topic = "people"
target_country_male_child_name = "Miguel"
target_country_female_child_name = "Alice"
target_country_population = "209 million"

# index row in selected_indicators; needs to be selected via target_topic?
temp_index = 0

#print (user_name)
#print (user_sex)
#print (user_country)
#print (target_country)
#print (target_topic)
#print (temp_index)

In [14]:
# Set up default derivative variables from selected_indicators

temp_priority_rank =              selected_indicators.loc[temp_index,"priority_rank"]
temp_priority_order =             selected_indicators.loc[temp_index,"priority_order"]
temp_indicator_id =               selected_indicators.loc[temp_index,"indicator_id"] 
temp_series_code =                selected_indicators.loc[temp_index,"series_code"]
temp_age_group =                  selected_indicators.loc[temp_index,"age_group"]
temp_location =                   selected_indicators.loc[temp_index,"location"]
temp_sex =                        selected_indicators.loc[temp_index,"sex"]
temp_value_type =                 selected_indicators.loc[temp_index,"value_type"]
temp_unit =                       selected_indicators.loc[temp_index,"unit"]
temp_parse_string_for_indicator = selected_indicators.loc[temp_index,"parse_string_for_indicator"]
temp_percent_divisor =            selected_indicators.loc[temp_index,"percent_divisor"]
temp_value_judgement =            selected_indicators.loc[temp_index,"value_judgement"]

#print ("temp_priority_rank =", temp_priority_rank)
#print ("temp_priority_order =", temp_priority_order)
#print ("temp_indicator_id =", temp_indicator_id)
#print ("temp_series_code =", temp_series_code)
#print ("temp_age_group =", temp_age_group)
#print ("temp_location =", temp_location)
#print ("temp_sex =", temp_sex)
#print ("temp_value_type =", temp_value_type)
#print ("temp_unit =", temp_unit)
#print ("temp_parse_string_for_indicator =", temp_parse_string_for_indicator)
#print ("temp_percent_divisor =", temp_percent_divisor)
#print ("temp_value_judgement =", temp_value_judgement)

In [15]:
# Extract row (not rows) from unsdg_data_exna
# rather than using logical.and, I will use divide and conquer using
# df2[df2['id'].isin(['SP.POP.TOTL','NY.GNP.PCAP.CD'])]

#print ("whole data set rows =", len(unsdg_data_exna.index))

temp_output_rows = unsdg_data_exna[unsdg_data_exna["country.or.area.name"].isin([target_country])]

#print ("target country rows =", len(temp_output_rows.index))

temp_output_rows = temp_output_rows[temp_output_rows["indicatorid"].isin([temp_indicator_id])]

#print ("target indicator id rows =", len(temp_output_rows.index))

temp_output_rows = temp_output_rows[temp_output_rows["series.code"].isin([temp_series_code])]

#print ("target series code rows =", len(temp_output_rows.index))

temp_output_rows = temp_output_rows[temp_output_rows["age.group"].isin([temp_age_group])]

#print ("target age group rows =", len(temp_output_rows.index))

temp_output_rows = temp_output_rows[temp_output_rows["location"].isin([temp_location])]

#print ("target location rows =", len(temp_output_rows.index))

temp_output_rows = temp_output_rows[temp_output_rows["sex"].isin([temp_sex])]

#print ("target sex rows =", len(temp_output_rows.index))

temp_output_rows = temp_output_rows[temp_output_rows["unit"].isin([temp_unit])]

#print ("target unit rows =", len(temp_output_rows.index))

# All value.type(s) we want to pull are null
temp_output_rows = temp_output_rows[temp_output_rows["value.type"].isnull()]

#print ("target value type rows =", len(temp_output_rows.index))

temp_count_rows = len(temp_output_rows.index)

print ("temp_count_rows =", temp_count_rows)

temp_output_rows.head(temp_count_rows)

temp_count_rows = 5


,Unnamed: 0,goal,target,indicator.ref,indicatorid,indicator.description,series.code,series.type,series.description,parent.country.or.area.code,...,source.type,age.group,location,sex,value.type,unit,unit.multiplier,year,value,footnote
51624,241265,8,8.6,8.6.1,C080601,Proportion of youth (aged 15-24 years) not in ...,SL_TLF_NEET,SD,"Proportion of youth not in education, employme...",M49_LAC_SA,...,Not available,15 to 24 years old,Total (national level),Both sexes or no breakdown by sex,NaN,Percent,Units,2009,18.42,"C, 20, 116, 229"
51978,242223,8,8.6,8.6.1,C080601,Proportion of youth (aged 15-24 years) not in ...,SL_TLF_NEET,SD,"Proportion of youth not in education, employme...",M49_LAC_SA,...,Not available,15 to 24 years old,Total (national level),Both sexes or no breakdown by sex,NaN,Percent,Units,2011,19.03,"C, 22, 116, 229"
52152,242702,8,8.6,8.6.1,C080601,Proportion of youth (aged 15-24 years) not in ...,SL_TLF_NEET,SD,"Proportion of youth not in education, employme...",M49_LAC_SA,...,Not available,15 to 24 years old,Total (national level),Both sexes or no breakdown by sex,NaN,Percent,Units,2012,18.93,"C, 23, 116, 229"
52337,243181,8,8.6,8.6.1,C080601,Proportion of youth (aged 15-24 years) not in ...,SL_TLF_NEET,SD,"Proportion of youth not in education, employme...",M49_LAC_SA,...,Not available,15 to 24 years old,Total (national level),Both sexes or no breakdown by sex,NaN,Percent,Units,2013,19.62,"C, 24, 116, 229"
52516,243660,8,8.6,8.6.1,C080601,Proportion of youth (aged 15-24 years) not in ...,SL_TLF_NEET,SD,"Proportion of youth not in education, employme...",M49_LAC_SA,...,Not available,15 to 24 years old,Total (national level),Both sexes or no breakdown by sex,NaN,Percent,Units,2014,19.28,"C, 25, 116, 229"


In [16]:
# Select correct first and last year results
# Assumes temp_count_rows should be an integer
# Assumes values can be converted to floating point numbers
# Don't need to re-pull parse string, already have it
# Omit conversion to % when compiling

#temp_count_rows = "test"

try:
    temp_count_rows = int(temp_count_rows)
except ValueError:
    temp_count_rows = -1
    
if temp_count_rows > 1:
    temp_count_of_results = "at least two results"
    print("temp_count_of_results =", temp_count_of_results)
    #find min and max years
    temp_max_year = max(temp_output_rows.year)
    temp_min_year = min(temp_output_rows.year)
    print ("temp max year =", temp_max_year)
    print ("temp min year =", temp_min_year)
    #find min and max years' values
    temp_max_year_value = temp_output_rows.value[temp_output_rows["year"].isin([temp_max_year])]
    temp_min_year_value = temp_output_rows.value[temp_output_rows["year"].isin([temp_min_year])]
    temp_max_year_value = float (temp_max_year_value.values[0]) / 100
    temp_min_year_value = float (temp_min_year_value.values[0]) / 100
    #find max v min year' values comparison value
    temp_comparison = temp_max_year_value / temp_min_year_value - 1
    print ("temp max year value =", temp_max_year_value)
    print ("temp min year value =", temp_min_year_value)
    print ("temp comparison =", temp_comparison)
elif temp_count_rows == 1: 
    temp_count_of_results = "one result"
    print("temp_count_of_results =", temp_count_of_results)
    #find min and max years
    temp_max_year = max(temp_output_rows.year)
    temp_min_year = np.nan
    print ("temp max year =", temp_max_year)
    print ("temp min year =", temp_min_year)
    #find min and max years' values
    temp_max_year_value = temp_output_rows.value[temp_output_rows["year"].isin([temp_max_year])]
    temp_min_year_value = np.nan
    temp_max_year_value = float (temp_max_year_value.values[0])
    temp_min_year_value = np.nan
    #find max v min year' values comparison value
    temp_comparison = np.nan
    print ("temp max year value =", temp_max_year_value)
    print ("temp min year value =", temp_min_year_value)
    print ("temp comparison =", temp_comparison)
elif temp_count_rows == 0:
    temp_count_of_results = "zero results"
    print("temp_count_of_results =", temp_count_of_results)
    temp_max_year = np.nan
    temp_min_year = np.nan
    temp_comparison = np.nan
else:
    temp_count_of_results = "zero results"
    print("temp_count_of_results =", temp_count_of_results)
    temp_max_year = np.nan
    temp_min_year = np.nan
    temp_comparison = np.nan

temp_count_of_results = at least two results
temp max year = 2014
temp min year = 2009
temp max year value = 0.1928
temp min year value = 0.18420000000000003
temp comparison = 0.046688382193267985


In [17]:
# Select correct percent and comparison descriptors

#temp_max_year_value = "test"
#temp_comparison = "test"

try:
    temp_max_year_value = float(temp_max_year_value)
except ValueError:
    temp_max_year_value = 0

if temp_max_year_value < 0.00:
    temp_percent_text_desc = "none"
elif temp_max_year_value < 0.05:
    temp_percent_text_desc = "almost none"
elif temp_max_year_value < 0.34:
    temp_percent_text_desc = "some"
elif temp_max_year_value < 0.68:
    temp_percent_text_desc = "about half"
elif temp_max_year_value < 0.95:
    temp_percent_text_desc = "many"
elif temp_max_year_value < 1.00:
    temp_percent_text_desc = "almost all"
else:
    temp_percent_text_desc = "all"

try:
    temp_comparison = float(temp_comparison)
except ValueError:
    temp_comparison = 0    
    
if temp_comparison < -1:
    temp_comparison_text_desc = "very much less than"  
elif temp_comparison < -0.2:
    temp_comparison_text_desc = "much less than"  
elif temp_comparison < -0.1:
    temp_comparison_text_desc = "less than"  
elif temp_comparison < 0.1:
    temp_comparison_text_desc = "about the same as"  
elif temp_comparison < 0.2:
    temp_comparison_text_desc = "greater than"  
elif temp_comparison < 1:
    temp_comparison_text_desc = "much greater than"  
else:
    temp_comparison_text_desc = "very much greater than"  

print("temp_max_year_value =", temp_max_year_value)
print("temp_percent_text_desc =", temp_percent_text_desc)
print("temp_comparison =", temp_comparison)
print("temp_comparison_text_desc  =", temp_comparison_text_desc) 


temp_max_year_value = 0.1928
temp_percent_text_desc = some
temp_comparison = 0.046688382193267985
temp_comparison_text_desc  = about the same as


In [18]:
# Output parsed results -- page one and page two results
# Assumes user_name, user_sex, target_country, target_country_male_child_name, 
#  target_country_female_child_name, and target_country_population are populated / not null
#  *** This code does not error catch where any of these variables are null *** 

#user_name = "test"
#user_sex = "test"

#temp user name
if user_name.strip() == "":
    temp_user_name = ""
else:
    temp_user_name = user_name.lower().capitalize().strip()
    
#print (user_name, " > ", temp_user_name)    

#hi padding
if user_name == "":
    hi_padding = ""
else:
    hi_padding =", "

#penpal name
if user_sex.lower() == "male":
    penpal_name = target_country_male_child_name
else:
    penpal_name = target_country_female_child_name

#did padding
if user_name == "":
    did_padding = "Did "
else:
    did_padding =", did "

#same country or not?
if user_country == target_country:
    same_country_padding = ", too! "
else:
    same_country_padding = ". "
    
# Page one section one
P1S1 = "Hi" + hi_padding + temp_user_name + "! My name is "+ penpal_name + ", and I am from " + target_country + same_country_padding
print (P1S1)
print ()

# Page one section two
P1S2 = "< *** insert first tranche of simple english wikipedia entries here *** >"
print (P1S2)
print ()

# Page two section one
P2S1 = temp_user_name + did_padding + "you known that about " + target_country_population + " people live in " + target_country + "? "
print (P2S1)
print ()

# Page two section two
P2S2 = "< *** insert second tranche of simple english wikipedia entries here *** >"
print (P2S2)
print ()


Hi, Alex! My name is Miguel, and I am from Brazil. 

< *** insert first tranche of simple english wikipedia entries here *** >

Alex, did you known that about 209 million people live in Brazil? 

< *** insert second tranche of simple english wikipedia entries here *** >



In [19]:
#temp_max_year_value rendered as a two digit percent string

temp_percent = str(int(round(temp_max_year_value * 100, 0))) + "%"

#print (temp_percent)
#print()
#print (temp_count_of_results)

19%

at least two results


In [20]:
# Output parsed results -- by topic results

#temp_count_of_results = "test"

# Topic page section one
if temp_count_of_results == "at least two results":
    #print (temp_count_of_results)
    #print()
    P3S1 = temp_user_name + did_padding + "you known that in " + target_country + " in " + str(int(temp_max_year)) + ", " + temp_percent_text_desc + " (" + temp_percent + ") " + temp_parse_string_for_indicator + " "
    P3S1 = P3S1 + "This is " + temp_comparison_text_desc + " in " + str(int(temp_min_year)) + ". "
elif temp_count_of_results == "one result":
    #print (temp_count_of_results)
    #print()
    P3S1 = temp_user_name + did_padding + "you known that in " + target_country + " in " + str(int(temp_max_year)) + ", " + temp_percent_text_desc + " (" + temp_percent + ") " + temp_parse_string_for_indicator + " "
else:
    #print (temp_count_of_results)
    #print()
    P3S1 = temp_user_name + hi_padding + "I am interested in " + target_topic + " too. I hope the people of " + user_country + " and " + target_country + " can be friends. "

print (P3S1)


Alex, did you known that in Brazil in 2014, some (19%) of its people aged 15 to 24 were not in school, education, or training, and also could not find a job? This is about the same as in 2009. 
